# Mixed Integer Linear Programming

Which is sort of famous, for valid reasons

## Mixed Integer Linear Programming

**A _Mixed Integer Linear Program_ is a problem in the form**

$$
\text{argmin}_x \left\{ c^Tx \mid Ax \geq b, x \geq 0, x_{I} \in \mathbb{Z} \right\}
$$

* The cost function and all constraints are linear
* All variables are non-negative
* _Some_ variables (those with index in $I$) are _integer_

**MILP is an extremely powerful formalism**

Thanks to the presence of integer variables

* ...Any _combinatorial element_ can be modeled
* ...And _non-linearity_ can be approximated

**MILP solvers classically rely on _three main techniques_**

## Linear Relaxation

**If we remove the integrality constraints from a MILP we obtain an LP**

<center><img src="assets/milp_geometric.png" width=700px/></center>

$$
\text{argmin}_x \left\{ c^Tx \mid Ax \geq b, x \geq 0 \right\}
$$

**This is called the _linear (or LP) relaxation_ of the MILP**

* The feasible space is defined via linear constraints $\Rightarrow$ is is a _polytope_
* The cost vector $c$ is also the _gradient_ and determines an optimization direction

## Linear Relaxation

**If we remove the integrality constraints from a MILP we obtain an LP**

<center><img src="assets/milp_simplex.png" width=700px/></center>

$$
\text{argmin}_x \left\{ c^Tx \mid Ax \geq b, x \geq 0 \right\}
$$

**LPs can be solved in pseudo-polynomial time via the [_Simplex method_](https://onlinelibrary.wiley.com/doi/book/10.1002/9781118627372)**

* The method start from a polytope vertex
* ...And then moves between adjiacent vertexes until the optimum is reached

## Linear Relaxation

**If we remove the integrality constraints from a MILP we obtain an LP**

<center><img src="assets/milp_simplex.png" width=700px/></center>

$$
\text{argmin}_x \left\{ c^Tx \mid Ax \geq b, x \geq 0 \right\}
$$

**LPs can be solved in _polynomial_ time via [Interior Point methods](https://web.stanford.edu/~boyd/cvxbook/)**

* These used to be slower in practice than the Simplex, but not anymore
* In a MILP complex, the Simplex is still preferred (later we will see why)

## Technique #1: Branching

**When tackling a MILP, we start by _solving its LP relaxation_**

<center><img src="assets/milp_branching.png" width=700px/></center>

* If all integrality constraints are satisfied, we have found the true optimum
* If some $x_j$ has a fraction value $v_j$, we _split the problem_ in two:
  - In the first subproblem, we add the constraint $x_j \leq \lfloor v_j \rfloor$
  - In the second subproblem, we add $x_j \geq \lceil v_j \rceil$
* Then we can repeat the whole process

## Technique #1: Branching

**This approach is referred to as _branching_**

<center><img src="assets/milp_branching.png" width=700px/></center>

* The first subproblem is also known as the _left branch_
* The second as the _right branch_

**Branching is the main method that makes MILP solvers complete**

* In the worst case, we end up with a _search tree_ having an exponential number of nodes
* ...But that's somewhat unavoidable, since _solving MILP is NP-hard_


## Technique #1: Branching

**This approach is referred to as _branching_**

<center><img src="assets/milp_branching.png" width=700px/></center>

**Branching is also the reason why the Simplex method is preferred to MILPs**

* The Simplex method has a "dual" version
* ...Whose optimum can be updated efficiently when new constraints are added

...And you can guess that's a pretty common operation ;-)

## Technique #2: Bounding

**Let's look again at the LP relaxation**

$$
\text{argmin}_x \left\{ c^Tx \mid Ax \geq b, x \geq 0 \right\}
$$

* The problem has the same structure
* ...But a larger feasible space (that's why it is called a relaxation)

Hence, its optimal cost will be a _lower bound_ (say $lb$) for the MILP

**We can use this bound as an early stopping criterion**

* Let $x^*$ be the best (mixed-integer) solution we have found so far
* If for some node of the search tree we have $lb > c^T x^*$
* Then we have no hope of beating $x^*$ and we can destroy (_fathom_) the node

**Branching + Bounding = Branch & Bound**

## Technique #3: Cutting Planes

**It is also common to speed-up MILP solution by using _cutting planes_**

<center><img src="assets/milp_cuts.png" width=700px/></center>

**Cutting planes are linear inequalities _inferred_ by relying on some property**

* In MILP they are typically inferred based on integrality constraint
* They must be _valid_ for any feasible solution
* They are useful if they force a fractional solution to become closer to integer

## Technique #3: Cutting Planes

**A common example is that of _Gomory Cuts_**

While solving the simplex, we end up with many equalities in the form:
$$
x_i + \sum_{j \in \bar{B}} \bar{a}_{ij} x_j = \bar{b}_i
$$

* Where $x_i > 0$ and $x_j = 0, \forall j \in \bar{B}$
* $B = $ the set of indexes of _non-zero_ variables in the current LP solution (base)
* $\bar{B} = $ the set of indexes of _zero_ variables in the current LP solution
* We will assume all variables are integer, for simplicity

**We can rewrite the equation as**

$$
x_i + \sum_{j \in \bar{B}} (\bar{a}_{ij} - \lfloor \bar{a}_{ij} \rfloor + \lfloor \bar{a}_{ij} \rfloor) x_j = \bar{b}_i - \lfloor \bar{b}_i \rfloor + \lfloor \bar{b}_i \rfloor
$$


## Technique #3: Cutting Planes

**By simple algebraic manipulation we can then get:**

$$
x_i + \sum_{j \in \bar{B}} \lfloor \bar{a}_{ij} \rfloor x_j - \lfloor \bar{b}_i \rfloor = - \sum_{j \in \bar{B}} (\bar{a}_{ij} - \lfloor \bar{a}_{ij} \rfloor) x_j + (\bar{b}_i - \lfloor \bar{b}_i \rfloor)
$$

We will build an inequality that is valid for _any feasible, integer_ point:

* The right-most part is necessarily $< 1$, since:
  - $\bar{b}_i - \lfloor \bar{b}_i \rfloor$ is positive and fractional
  - Each $\bar{a}_{ij} - \lfloor \bar{a}_{ij} \rfloor$ is positive (and fractional)
  - Each $x_j$ must be $\geq 0$
* The left-most part is necessarily an integer, since:
  - $\lfloor \bar{b}_i \rfloor$ is integer and each $\lfloor \bar{a}_{ij} \rfloor$ is integer
  - Variables are integer as per our assumption

## Technique #3: Cutting Planes

**By simple algebraic manipulation we can then get:**

$$
x_i + \sum_{j \in \bar{B}} \lfloor \bar{a}_{ij} \rfloor x_j - \lfloor \bar{b}_i \rfloor = - \sum_{j \in \bar{B}} (\bar{a}_{ij} - \lfloor \bar{a}_{ij} \rfloor) x_j + (\bar{b}_i - \lfloor \bar{b}_i \rfloor)
$$

* Hence, the right-most part should be $< 1$ and integer
* ...Meaning that it must be $\leq 0$

$$
- \sum_{j \in \bar{B}} (\bar{a}_{ij} - \lfloor \bar{a}_{ij} \rfloor) x_j + (\bar{b}_i - \lfloor \bar{b}_i \rfloor) \leq 0
$$

And from here:

$$
\sum_{j \in \bar{B}} (\bar{a}_{ij} - \lfloor \bar{a}_{ij} \rfloor) x_j \geq (\bar{b}_i - \lfloor \bar{b}_i \rfloor)
$$


## Technique #3: Cutting Planes

**This inequality is the Gomory Cut**

$$
\sum_{j \in \bar{B}} (\bar{a}_{ij} - \lfloor \bar{a}_{ij} \rfloor) x_j \geq (\bar{b}_i - \lfloor \bar{b}_i \rfloor)
$$

* Now, if we target a $x_i$ that should be integer, i.e. $i \in I$
* ...But it's fractional in the current solution

Then we have $\bar{b}_i - \lfloor \bar{b}_i \rfloor > 0$

* Combined with the fact that $x_j = 0, \forall j \in \bar{B}$ in the current solution
* We have that the cut is actually _making the solution no longer feasible_

**Branching + Bounding + Cutting Planes = Branch & Cut**

* Using cutting planes can speed up the solution process considerably
* But it's best not to overdo it, since subsequent cuts may become weaker

## Some Considerations

**We have just scratched the surface with MILP**

Modern MILP solver do much more:

* Presolving
* Constraint propagations
* Symmetry breaking
* ...

**MILP methods have a long history**

* There is a _huge gap_ between the solver performance
* The best solvers ([Gurobi](https://www.gurobi.com/), [Cplex](https://www.ibm.com/analytics/cplex-optimizer), [Mosek](https://www.mosek.com/)) are commercial (free for academics)
* Then you have a single semi-free solver ([SCIP](https://www.scipopt.org/))
* ...A good free solver ([CBC](https://github.com/coin-or/Cbc))
* ...And finally there is stuff you _should not_ touch (glpk, lpsolve)